In [1]:
from datetime import datetime
import pytz
import os
import time
import requests
import json

import pickle
from pathlib import Path
import traceback
from faker import Faker

fake = Faker()

In [2]:
# game_steamid = 1716740              # starfield
# game_name = 'starfield'             # also the folder name where the reviews are stored

# game_steamid = 1118010
# game_name = 'monster_hunter_world_iceborne'

game_steamid = 582010
game_name = 'monster_hunter_world'


In [3]:
# load the reviews from folder

reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'./{game_name}')
if game_folder.exists():
    try:
        latest_file_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique.pkl')
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            print('Loaded:', latest_file_path)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()

Loaded: monster_hunter_world/steam_reviews_582010_unique.pkl


In [4]:
# load the user stats (prevent reusing the same user)
# as we need each steamid -> unique user on our platform
user_stats = {}
user_stats_path = game_folder.parent.joinpath(f'user_stats_cross-game.pkl')
if user_stats_path.exists():
    with open(user_stats_path, 'rb') as f:
        user_stats = pickle.load(f)
        print('Loaded:', user_stats_path)

Loaded: user_stats_cross-game.pkl


In [5]:
len(reviews_reqs)

143026

In [7]:
reviews_reqs[-923]

{'recommendationid': '44029721',
 'author': {'steamid': '76561198005319999',
  'num_games_owned': 0,
  'num_reviews': 5,
  'playtime_forever': 15870,
  'playtime_last_two_weeks': 0,
  'playtime_at_review': 1558,
  'last_played': 1682956712},
 'language': 'english',
 'review': "MH4U was my last played, but I skipped Generations due to burning out - it was worth the wait! I loved Monster Hunter before I even played World, so this was a 'Have to buy' for me.\n\nGraphics are amazing, the sound is crisp, and the gameplay is exactly how it should be. \n\nThe story is actually engaging for a monster hunter game! As much as this game is the embodiment of grind (see: Warframe also) having a story is hardly a big selling point, however, it now adds to the already amazing gameplay.\n\n10/10 will grind 100+ hours in the release week - I took time off for this!",
 'timestamp_created': 1533854321,
 'timestamp_updated': 1533854321,
 'voted_up': True,
 'votes_up': 0,
 'votes_funny': 0,
 'weighted_vote

In [16]:
import random
random.seed(4801)

ip = 'https://critiqbackend.itzjacky.info'
port = 9000

add_review_url = f'{ip}:{port}/api/review/addReview'

# available_user_ids = [42, 43, 44]
# counter = 319       # Monster Hunter World counter

backend_json_template = {
    "score": 80,
    "comment": "Pretty good demo showcase of the game features!",
    "gameId": 7,
    "platform": "STEAM",                # platform must be in that game's platform list
    "reviewerId": 6,
    "recommended": True,
    "isSponsored": False,               # if true, show the msg for game recv for free
    "playTime": 400,                     # minutes
    "gameVersion": "2",
    "createdAt": "2018-12-30 11:11:11"  # UTC time
}

backend_adduser_template = {
    "name": "aaa",
    "email": "aaa@aaa.com",
    "password": "aaa",
    "birthday": "1999-07-03",
    "gender": "MALE"        # "MALE" or "FEMALE"
}

game_id_on_fyp_platform = 4     # Monster Hunter World

In [18]:
for i, review in enumerate(reviews_reqs):
    if i <= counter:
        continue

    try:
        json_body = backend_json_template.copy()
        user_json_body = backend_adduser_template.copy()

        # check if the user is already in the user_stats
        if reviews_reqs[i]['author']['steamid'] in user_stats:
            reviewer_id = user_stats[reviews_reqs[i]['author']['steamid']]['id']        # use the id from our platform
            json_body['reviewerId'] = reviewer_id
            print('User found:', reviewer_id)
        else:
            retry_counter = 0
            while retry_counter < 10:
                # add the user to the user_stats
                username = fake.user_name()
                while len(username) > 14:
                    username = fake.user_name()
                remaining_length = 14 - len(username)
                # generate username, with combination of name and random number, until 14 characters
                username = username + str(fake.random_number(min(remaining_length, 4)))     # add random number to the end
                # let the password be the same as the username
                password = username
                # generate email address from username
                # email address from github copilot suggestion
                email = username + random.choice(["@gmail.com", "@yahoo.com", "@hotmail.com", "@outlook.com", "@aol.com", "@protonmail.com", "@tutanota.com", "@mail.com", "@yandex.com", "@zoho.com", "@icloud.com", "@gmx.com", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru", "@mail.ru", "@rambler.ru", "@bk.ru", "@list.ru", "@inbox.ru", "@yandex.ru", "@ya.ru", "@narod.ru", "@ro.ru", "@rbcmail.ru", "@km.ru", "@ngs.ru"])
                # generate birthdate
                birthdate = fake.date_of_birth(minimum_age=13, maximum_age=99)
                # generate gender
                gender = random.choices(['MALE', "FEMALE"], weights=[0.7, 0.3], k=1)[0]

                user_json_body['name'] = username
                user_json_body['email'] = email
                user_json_body['password'] = password
                user_json_body['birthday'] = birthdate.strftime('%Y-%m-%d')
                user_json_body['gender'] = gender

                
                print(user_json_body)

                # first create user
                response = requests.post(f'{ip}:{port}/api/user/addUser', json=user_json_body)
                print(response.status_code)
                if response.status_code != 200:
                    print('Error:', response.text)
                    print('Retry to create a diff user'); retry_counter += 1
                else:
                    break

            if response.status_code != 200:
                print('Error:', response.text)
                print('Counter:', counter)
                break

            # store the user_stats
            user_stats[reviews_reqs[i]['author']['steamid']] = response.json()

        # then create the review
        review_text = review['review']
        received_for_free = review['received_for_free']
        playtime = review['author']['playtime_at_review'] if 'playtime_at_review' in review['author'] else review['author']['playtime_forever']       # the original playtime is in minutes also
        reviewer_id = user_stats[reviews_reqs[i]['author']['steamid']]['id']        # use the id from our platform
        game_version = i + 1
        score = random.randint(51, 95) if review['voted_up'] else random.randint(10, 50)
        dt = pytz.timezone('Asia/Hong_Kong').localize(datetime.fromtimestamp(review['timestamp_created'])).astimezone(pytz.utc)

        json_body['score'] = score
        json_body['comment'] = review_text
        json_body['gameId'] = game_id_on_fyp_platform
        json_body['reviewerId'] = reviewer_id
        json_body['recommended'] = review['voted_up']
        json_body['isSponsored'] = received_for_free
        json_body['playTime'] = playtime
        json_body['gameVersion'] = "1"
        json_body['createdAt'] = dt.strftime('%Y-%m-%d %H:%M:%S')
        print(json_body)

        response = requests.post(add_review_url, json=json_body)
        print(response.status_code)
        print(response.text)
        counter += 1

        if response.status_code != 200:
            print('Error:', response.text)
            break
    except Exception as e:
        print('Error:', e)
        print('Count:', counter)
        traceback.print_exc()

    print('Counter:', counter)
    print()

    if i == 1000:
        break

{'name': 'harrissandra49', 'email': 'harrissandra49@ya.ru', 'password': 'harrissandra49', 'birthday': '1930-06-17', 'gender': 'MALE'}
200
{'score': 95, 'comment': 'Love it  !', 'gameId': 4, 'platform': 'STEAM', 'reviewerId': 128, 'recommended': True, 'isSponsored': False, 'playTime': 8666, 'gameVersion': '1', 'createdAt': '2024-02-24 10:56:32'}
200
{"id":418,"dislikedUsers":null,"likedUsers":null,"reviewer":{"id":128,"isPrivate":true,"isVerified":false,"name":"harrissandra49","email":"harrissandra49@ya.ru","birthday":"1930-06-17T00:00:00.000+00:00","age":"93","ageGroup":null,"gender":"MALE","joinDate":"Mon Feb 26 07:35:36 UTC 2024","lastActive":null,"favouriteGames":[],"wishlistGames":[],"numOfReviews":1,"role":["USER"],"reviews":null,"numberOfLikes":0,"numberOfDislikes":0,"numberOfReviews":0,"iconUrl":null,"bannerUrl":null,"enabled":true,"accountNonExpired":true,"accountNonLocked":true,"credentialsNonExpired":true,"username":"harrissandra49@ya.ru","authorities":[{"authority":"ROLE_USE

KeyboardInterrupt: 

In [20]:
# save the user stats
with open(user_stats_path, 'wb') as f:
    pickle.dump(user_stats, f)
    print('Saved:', user_stats_path)

Saved: user_stats_cross-game.pkl


In [19]:
counter

319